In [8]:
import os
import sys
import numpy as np
import pandas as pd

import SimpleITK as sitk

from multiprocessing import Pool
import multiprocessing as mp

In [9]:
phase1steps_path = os.path.join('SCCOR_steps_number.csv')

phase1steps_df = pd.read_csv(phase1steps_path,index_col='subj')
phase1steps_df.head()

,#dicom,#niix,#iso,#partial,#partialUnet,#lobeUnet,#mask,#maskUnet,#supervoxel,#fHOG,#fHOG_Hist,#Haralick_feature
subj,,,,,,,,,,,,
1-0594-2,1,1,0,0,0,0,0,0,0,0,0,0
1-0594-2,501,501,501,0,501,501,0,0,0,0,0,0
1-0594-2,126,126,126,0,126,126,0,0,0,0,0,0
1-0594-2,126,126,126,0,126,126,0,0,0,0,0,0
1-0682-6,61,61,61,0,61,61,0,0,0,0,0,0


In [10]:
# Progress Status
# 3 = queue
# 2 = complete
# 1 = partial
# 0 = failed

def count_progress(record_tuple):
    index,row = record_tuple
    
    num_orig_dicom,num_regen_niix,num_regen_iso, \
        num_regen_partial,num_regen_partial_unet,num_regen_lobe_unet,num_regen_mask, \
        num_regen_mask_unet,num_regen_supervoxel,num_regen_fHOG, \
        num_regen_fHOG_hist,num_regen_haralick_feature = row
    
    niix_status = 0
    iso_status = 0
    partial_status = 0
    partialUnet_status = 0
    lobeUnet_status = 0
    mask_status = 0
    maskUnet_status = 0
    supervoxel_status = 0
    fHOG_status = 0
    fHOG_hist_status = 0
    haralick_feature_status = 0
    
    if num_orig_dicom == num_regen_niix:
        niix_status = 2
    elif (niix_queue_subj is not None) and (niix_queue_subj <= index):
        niix_status = 3
    elif num_regen_niix > 0:
        niix_status = 1
        
    if num_orig_dicom == num_regen_iso:
        iso_status = 2
    elif (iso_queue_subj is not None) and (iso_queue_subj <= index):
        iso_status = 3
    elif num_regen_iso > 0:
        iso_status = 1

    if num_orig_dicom == num_regen_partial:
        partial_status = 2
    elif (partial_queue_subj is not None) and (partial_queue_subj <= index):
        partial_status = 3
    elif num_regen_partial > 0:
        partial_status = 1
    
    if num_orig_dicom == num_regen_partial_unet:
        partialUnet_status = 2
    elif (partialUnet_queue_subj is not None) and (partialUnet_queue_subj <= index):
        partialUnet_status = 3
    elif num_regen_partial_unet > 0:
        partialUnet_status = 1
        
    if num_orig_dicom == num_regen_lobe_unet:
        lobeUnet_status = 2
    elif (lobeUnet_queue_subj is not None) and (lobeUnet_queue_subj <= index):
        lobeUnet_status = 3
    elif num_regen_lobe_unet > 0:
        lobeUnet_status = 1
        
    if num_orig_dicom == num_regen_mask:
        mask_status = 2
    elif (mask_queue_subj is not None) and (mask_queue_subj <= index):
        mask_status = 3
    elif num_regen_mask > 0:
        mask_status = 1
        
    if num_orig_dicom == num_regen_mask_unet:
        maskUnet_status = 2
    elif (maskUnet_queue_subj is not None) and (maskUnet_queue_subj <= index):
        maskUnet_status = 3
    elif num_regen_mask_unet > 0:
        maskUnet_status = 1
        
    if num_orig_dicom == num_regen_supervoxel:
        supervoxel_status = 2
    elif (supervoxel_queue_subj is not None) and (supervoxel_queue_subj <= index):
        supervoxel_status = 3
    elif num_regen_supervoxel > 0:
        supervoxel_status = 1
        
    if num_orig_dicom == num_regen_fHOG:
        fHOG_status = 2
    elif (fHOG_queue_subj is not None) and (fHOG_queue_subj <= index):
        fHOG_status = 3
    elif num_regen_fHOG > 0:
        fHOG_status = 1
     
    if num_orig_dicom == num_regen_fHOG_hist:
        fHOG_hist_status = 2
    elif (fHOG_hist_queue_subj is not None) and (fHOG_hist_queue_subj <= index):
        fHOG_hist_status = 3
    elif num_regen_fHOG_hist > 0:
        fHOG_hist_status = 1
    
    if num_orig_dicom == num_regen_haralick_feature:
        haralick_feature_status = 2
    elif (haralick_feature_queue_subj is not None) and (fHOG_hist_queue_subj <= index):
        haralick_feature_status = 3
    elif num_regen_haralick_feature > 0:
        haralick_feature_status = 1
    
    return (niix_status,iso_status,partial_status,partialUnet_status,lobeUnet_status,mask_status, \
            maskUnet_status,supervoxel_status,fHOG_status,fHOG_hist_status,haralick_feature_status)

In [11]:
niix_queue_subj = None
iso_queue_subj = None
partial_queue_subj = None
partialUnet_queue_subj = None
lobeUnet_queue_subj = None
mask_queue_subj = None
maskUnet_queue_subj = None
supervoxel_queue_subj = None
fHOG_queue_subj = None
fHOG_hist_queue_subj = None
haralick_feature_queue_subj = None

In [12]:
processes = mp.cpu_count()
pool = Pool(processes=processes)

# Calculate a good chunksize (based on implementation of pool.map)
chunksize, extra = divmod(phase1steps_df.shape[0], 4 * processes)
if extra:
    chunksize += 1

results = [result for result in pool.imap_unordered(count_progress,
                                    ((index,row) for index,row in phase1steps_df.iterrows()),
                                    chunksize=chunksize)]

pool.close()
pool.join()

In [13]:
total_num_phase1_subjs = phase1steps_df.shape[0]

total_num_phase1_complete_niix = 0
total_num_phase1_complete_iso = 0
total_num_phase1_complete_partial = 0
total_num_phase1_complete_partial_unet = 0
total_num_phase1_complete_lobe_unet = 0
total_num_phase1_complete_mask = 0
total_num_phase1_complete_mask_unet = 0
total_num_phase1_complete_supervoxel = 0
total_num_phase1_complete_fHOG = 0
total_num_phase1_complete_fHOG_hist = 0
total_num_phase1_complete_haralick_feature = 0

total_num_phase1_queue_niix = 0
total_num_phase1_queue_iso = 0
total_num_phase1_queue_partial = 0
total_num_phase1_queue_partial_unet = 0
total_num_phase1_queue_lobe_unet = 0
total_num_phase1_queue_mask = 0
total_num_phase1_queue_mask_unet = 0
total_num_phase1_queue_supervoxel = 0
total_num_phase1_queue_fHOG = 0
total_num_phase1_queue_fHOG_hist = 0
total_num_phase1_queue_haralick_feature = 0

total_num_phase1_partial_niix = 0
total_num_phase1_partial_iso = 0
total_num_phase1_partial_partial = 0
total_num_phase1_partial_partial_unet = 0
total_num_phase1_partial_lobe_unet = 0
total_num_phase1_partial_mask = 0
total_num_phase1_partial_mask_unet = 0
total_num_phase1_partial_supervoxel = 0
total_num_phase1_partial_fHOG = 0
total_num_phase1_partial_fHOG_hist = 0
total_num_phase1_partial_haralick_feature = 0

for niix_status,iso_status,partial_status,partialUnet_status,lobeUnet_status,mask_status, \
            maskUnet_status,supervoxel_status,fHOG_status, \
            fHOG_hist_status,haralick_feature_status in results:
    
    if niix_status == 2:
        total_num_phase1_complete_niix += 1
    elif niix_status == 3:
        total_num_phase1_queue_niix += 1
    elif niix_status == 1:
        total_num_phase1_partial_niix += 1
        
    if iso_status == 2:
        total_num_phase1_complete_iso += 1
    elif iso_status == 3:
        total_num_phase1_queue_iso += 1
    elif iso_status == 1:
        total_num_phase1_partial_iso += 1
        
    if partial_status == 2:
        total_num_phase1_complete_partial += 1
    elif partial_status == 3:
        total_num_phase1_queue_partial += 1
    elif partial_status == 1:
        total_num_phase1_partial_partial += 1
        
    if partialUnet_status == 2:
        total_num_phase1_complete_partial_unet += 1
    elif partialUnet_status == 3:
        total_num_phase1_queue_partial_unet += 1
    elif partialUnet_status == 1:
        total_num_phase1_partial_partial_unet += 1
        
    if lobeUnet_status == 2:
        total_num_phase1_complete_lobe_unet += 1
    elif lobeUnet_status == 3:
        total_num_phase1_queue_lobe_unet += 1
    elif lobeUnet_status == 1:
        total_num_phase1_partial_lobe_unet += 1
        
    if mask_status == 2:
        total_num_phase1_complete_mask += 1
    elif mask_status == 3:
        total_num_phase1_queue_mask += 1
    elif mask_status == 1:
        total_num_phase1_partial_mask += 1
        
    if maskUnet_status == 2:
        total_num_phase1_complete_mask_unet += 1
    elif maskUnet_status == 3:
        total_num_phase1_queue_mask_unet += 1
    elif maskUnet_status == 1:
        total_num_phase1_partial_mask_unet += 1
        
    if supervoxel_status == 2:
        total_num_phase1_complete_supervoxel += 1
    elif supervoxel_status == 3:
        total_num_phase1_queue_supervoxel += 1
    elif supervoxel_status == 1:
        total_num_phase1_partial_supervoxel += 1
        
    if fHOG_status == 2:
        total_num_phase1_complete_fHOG += 1
    elif fHOG_status == 3:
        total_num_phase1_queue_fHOG += 1
    elif fHOG_status == 1:
        total_num_phase1_partial_fHOG += 1
        
    if fHOG_hist_status == 2:
        total_num_phase1_complete_fHOG_hist += 1
    elif fHOG_hist_status == 3:
        total_num_phase1_queue_fHOG_hist += 1
    elif fHOG_hist_status == 1:
        total_num_phase1_partial_fHOG_hist += 1
        
    if haralick_feature_status == 2:
        total_num_phase1_complete_haralick_feature += 1
    elif haralick_feature_status == 3:
        total_num_phase1_queue_haralick_feature += 1
    elif haralick_feature_status == 1:
        total_num_phase1_partial_haralick_feature += 1
        
print('total_num_phase1_subjs:',total_num_phase1_subjs)        

print('###############################################')      
print('total_num_phase1_complete_niix:',total_num_phase1_complete_niix)
print('total_num_phase1_complete_iso:',total_num_phase1_complete_iso)
print('total_num_phase1_complete_partial:',total_num_phase1_complete_partial)
print('total_num_phase1_complete_partial_unet:',total_num_phase1_complete_partial_unet)
print('total_num_phase1_complete_lobe_unet:',total_num_phase1_complete_lobe_unet)
print('total_num_phase1_complete_mask:',total_num_phase1_complete_mask)
print('total_num_phase1_complete_mask_unet:',total_num_phase1_complete_mask_unet)
print('total_num_phase1_complete_supervoxel:',total_num_phase1_complete_supervoxel)
print('total_num_phase1_complete_fHOG:',total_num_phase1_complete_fHOG)
print('total_num_phase1_complete_fHOG_hist:',total_num_phase1_complete_fHOG_hist)
print('total_num_phase1_complete_haralick_feature:',total_num_phase1_complete_haralick_feature)

print('###############################################')      
print('total_num_phase1_queue_niix:',total_num_phase1_queue_niix)
print('total_num_phase1_queue_iso:',total_num_phase1_queue_iso)
print('total_num_phase1_queue_partial:',total_num_phase1_queue_partial)
print('total_num_phase1_queue_partial_unet:',total_num_phase1_queue_partial_unet)
print('total_num_phase1_queue_lobe_unet:',total_num_phase1_queue_lobe_unet)
print('total_num_phase1_queue_mask:',total_num_phase1_queue_mask)
print('total_num_phase1_queue_mask_unet:',total_num_phase1_queue_mask_unet)
print('total_num_phase1_queue_supervoxel:',total_num_phase1_queue_supervoxel)
print('total_num_phase1_queue_fHOG:',total_num_phase1_queue_fHOG)     
print('total_num_phase1_queue_fHOG_hist:',total_num_phase1_queue_fHOG_hist)     
print('total_num_phase1_queue_haralick_feature:',total_num_phase1_queue_haralick_feature)

print('###############################################')      
print('total_num_phase1_partial_niix:',total_num_phase1_partial_niix)
print('total_num_phase1_partial_iso:',total_num_phase1_partial_iso)
print('total_num_phase1_partial_partial:',total_num_phase1_partial_partial)
print('total_num_phase1_partial_partial_unet:',total_num_phase1_partial_partial_unet)
print('total_num_phase1_partial_lobe_unet:',total_num_phase1_partial_lobe_unet)
print('total_num_phase1_partial_mask:',total_num_phase1_partial_mask)
print('total_num_phase1_partial_mask_unet:',total_num_phase1_partial_mask_unet)
print('total_num_phase1_partial_supervoxel:',total_num_phase1_partial_supervoxel)
print('total_num_phase1_partial_fHOG:',total_num_phase1_partial_fHOG)
print('total_num_phase1_partial_fHOG_hist:',total_num_phase1_partial_fHOG_hist)
print('total_num_phase1_partial_haralick_feature:',total_num_phase1_partial_haralick_feature)

total_num_phase1_subjs: 5882
###############################################
total_num_phase1_complete_niix: 5690
total_num_phase1_complete_iso: 4299
total_num_phase1_complete_partial: 0
total_num_phase1_complete_partial_unet: 4299
total_num_phase1_complete_lobe_unet: 4299
total_num_phase1_complete_mask: 0
total_num_phase1_complete_mask_unet: 0
total_num_phase1_complete_supervoxel: 0
total_num_phase1_complete_fHOG: 0
total_num_phase1_complete_fHOG_hist: 0
total_num_phase1_complete_haralick_feature: 0
###############################################
total_num_phase1_queue_niix: 0
total_num_phase1_queue_iso: 0
total_num_phase1_queue_partial: 0
total_num_phase1_queue_partial_unet: 0
total_num_phase1_queue_lobe_unet: 0
total_num_phase1_queue_mask: 0
total_num_phase1_queue_mask_unet: 0
total_num_phase1_queue_supervoxel: 0
total_num_phase1_queue_fHOG: 0
total_num_phase1_queue_fHOG_hist: 0
total_num_phase1_queue_haralick_feature: 0
###############################################
total_num_phas

In [14]:
copd_steps_progress_columns = ['step','#img complete','#img partial','#img queue','#img failed','#img NA','#total',
                               '%complete','%partial','%queue','%failed','%NA','%total']
copd_phase1_steps_df = pd.DataFrame(columns = copd_steps_progress_columns)

phase1_num_failed_niix = total_num_phase1_subjs - total_num_phase1_complete_niix - total_num_phase1_queue_niix - total_num_phase1_partial_niix

copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['Dicom --> Niix',
     total_num_phase1_complete_niix,
     total_num_phase1_partial_niix, 
     total_num_phase1_queue_niix, 
     total_num_phase1_subjs - total_num_phase1_complete_niix - total_num_phase1_queue_niix - total_num_phase1_partial_niix, 
     0,
     total_num_phase1_subjs,
     
     total_num_phase1_complete_niix/total_num_phase1_subjs*100, 
     total_num_phase1_partial_niix/total_num_phase1_subjs*100, 
     total_num_phase1_queue_niix/total_num_phase1_subjs*100, 
     100.0*(1-(total_num_phase1_complete_niix+total_num_phase1_queue_niix+total_num_phase1_partial_niix)/total_num_phase1_subjs),
     0.0,
     100.0]

phase1_num_na_iso = total_num_phase1_queue_niix + phase1_num_failed_niix + 0
phase1_num_failed_iso = total_num_phase1_subjs - \
            (total_num_phase1_complete_iso + total_num_phase1_queue_iso + total_num_phase1_partial_iso + phase1_num_na_iso)

phase1_percent_complete_iso = total_num_phase1_complete_iso/total_num_phase1_subjs*100
phase1_percent_partial_iso = total_num_phase1_partial_iso/total_num_phase1_subjs*100
phase1_percent_queue_iso = total_num_phase1_queue_iso/total_num_phase1_subjs*100
phase1_percent_failed_iso = phase1_num_failed_iso/total_num_phase1_subjs*100.0
phase1_percent_na_iso = phase1_num_na_iso/total_num_phase1_subjs*100.0

copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['Niix --> Isotropic',
     total_num_phase1_complete_iso,
     total_num_phase1_partial_iso, 
     total_num_phase1_queue_iso, 
     phase1_num_failed_iso, 
     phase1_num_na_iso,
     total_num_phase1_complete_iso + total_num_phase1_queue_iso + total_num_phase1_partial_iso + phase1_num_failed_iso + phase1_num_failed_niix,
     
     phase1_percent_complete_iso, 
     phase1_percent_partial_iso, 
     phase1_percent_queue_iso, 
     phase1_percent_failed_iso,
     phase1_percent_na_iso,
     phase1_percent_complete_iso + phase1_percent_queue_iso + phase1_percent_partial_iso + phase1_percent_failed_iso + phase1_percent_na_iso
    ]

phase1_num_na_partial = total_num_phase1_queue_iso + phase1_num_failed_iso + phase1_num_na_iso
phase1_num_failed_partial = total_num_phase1_subjs - \
            (total_num_phase1_complete_partial + total_num_phase1_queue_partial + total_num_phase1_partial_partial + phase1_num_na_partial)

copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['Isotropic --> PartialLabelMap',
     total_num_phase1_complete_partial,
     total_num_phase1_partial_partial,
     total_num_phase1_queue_partial,
     phase1_num_failed_partial, 
     phase1_num_na_partial,
     total_num_phase1_complete_partial + total_num_phase1_queue_partial + total_num_phase1_partial_partial + 
         phase1_num_failed_partial + phase1_num_na_partial,
     
     total_num_phase1_complete_partial/total_num_phase1_subjs*100, 
     total_num_phase1_partial_partial/total_num_phase1_subjs*100, 
     total_num_phase1_queue_partial/total_num_phase1_subjs*100, 
     phase1_num_failed_partial/total_num_phase1_subjs*100,
     phase1_num_na_partial/total_num_phase1_subjs*100,
     total_num_phase1_complete_partial/total_num_phase1_subjs*100 + 
         total_num_phase1_queue_partial/total_num_phase1_subjs*100 + 
         total_num_phase1_partial_partial/total_num_phase1_subjs*100 + 
         phase1_num_failed_partial/total_num_phase1_subjs*100 + 
         phase1_num_na_partial/total_num_phase1_subjs*100
     ]

phase1_num_na_partial_unet = total_num_phase1_queue_iso + phase1_num_failed_iso + phase1_num_na_iso
phase1_num_failed_partial_unet = total_num_phase1_subjs - \
            (total_num_phase1_complete_partial_unet + total_num_phase1_queue_partial_unet + total_num_phase1_partial_partial_unet + phase1_num_na_partial_unet)

copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['Isotropic --> PartialLabelMap Unet',
     total_num_phase1_complete_partial_unet, 
     total_num_phase1_partial_partial_unet, 
     total_num_phase1_queue_partial_unet, 
     phase1_num_failed_partial_unet,
     phase1_num_na_partial_unet,
     total_num_phase1_complete_partial_unet +
         total_num_phase1_partial_partial_unet +
         total_num_phase1_queue_partial_unet +
         phase1_num_failed_partial_unet +
         phase1_num_na_partial_unet,
     
     total_num_phase1_complete_partial_unet/total_num_phase1_subjs*100, 
     total_num_phase1_partial_partial_unet/total_num_phase1_subjs*100, 
     total_num_phase1_queue_partial_unet/total_num_phase1_subjs*100, 
     phase1_num_failed_partial_unet/total_num_phase1_subjs*100,
     phase1_num_na_partial_unet/total_num_phase1_subjs*100,
     total_num_phase1_complete_partial_unet/total_num_phase1_subjs*100 + 
         total_num_phase1_partial_partial_unet/total_num_phase1_subjs*100 + 
         total_num_phase1_queue_partial_unet/total_num_phase1_subjs*100 + 
         phase1_num_failed_partial_unet/total_num_phase1_subjs*100 + 
         phase1_num_na_partial_unet/total_num_phase1_subjs*100
     ]

phase1_num_na_lobe_unet = total_num_phase1_queue_iso + phase1_num_failed_iso + phase1_num_na_iso
phase1_num_failed_lobe_unet = total_num_phase1_subjs - \
            (total_num_phase1_complete_lobe_unet + total_num_phase1_queue_lobe_unet + total_num_phase1_partial_lobe_unet + phase1_num_na_lobe_unet)

if phase1_num_failed_lobe_unet < 0:
    total_num_phase1_queue_lobe_unet += phase1_num_failed_lobe_unet
    phase1_num_failed_lobe_unet = 0

copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['Isotropic --> LobeSegmentation Unet',
     total_num_phase1_complete_lobe_unet,
     total_num_phase1_partial_lobe_unet,
     total_num_phase1_queue_lobe_unet,
     phase1_num_failed_lobe_unet,
     phase1_num_na_lobe_unet,
     total_num_phase1_complete_lobe_unet + total_num_phase1_queue_lobe_unet + total_num_phase1_partial_lobe_unet + phase1_num_failed_lobe_unet + phase1_num_na_lobe_unet,
     
     total_num_phase1_complete_lobe_unet/total_num_phase1_subjs*100, 
     total_num_phase1_partial_lobe_unet/total_num_phase1_subjs*100, 
     total_num_phase1_queue_lobe_unet/total_num_phase1_subjs*100, 
     phase1_num_failed_lobe_unet/total_num_phase1_subjs*100,
     phase1_num_na_lobe_unet/total_num_phase1_subjs*100,
     total_num_phase1_complete_lobe_unet/total_num_phase1_subjs*100 + 
         total_num_phase1_partial_lobe_unet/total_num_phase1_subjs*100 + 
         total_num_phase1_queue_lobe_unet/total_num_phase1_subjs*100 + 
         phase1_num_failed_lobe_unet/total_num_phase1_subjs*100 + 
         phase1_num_na_lobe_unet/total_num_phase1_subjs*100]

phase1_num_na_mask = total_num_phase1_queue_partial + phase1_num_failed_partial + phase1_num_na_partial
phase1_num_failed_mask = total_num_phase1_subjs - \
            (total_num_phase1_complete_mask + total_num_phase1_queue_mask + total_num_phase1_partial_mask + phase1_num_na_mask)

if phase1_num_failed_mask < 0:
    total_num_phase1_queue_mask += phase1_num_failed_mask
    phase1_num_failed_mask = 0

copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['PartialLabelMap --> Masking',
     total_num_phase1_complete_mask,
     total_num_phase1_partial_mask,
     total_num_phase1_queue_mask,
     phase1_num_failed_mask,
     phase1_num_na_mask,
     total_num_phase1_complete_mask + total_num_phase1_queue_mask + total_num_phase1_partial_mask + phase1_num_failed_mask + phase1_num_na_mask,
     
     total_num_phase1_complete_mask/total_num_phase1_subjs*100, 
     total_num_phase1_partial_mask/total_num_phase1_subjs*100, 
     total_num_phase1_queue_mask/total_num_phase1_subjs*100, 
     phase1_num_failed_mask/total_num_phase1_subjs*100,
     phase1_num_na_mask/total_num_phase1_subjs*100,
     
     total_num_phase1_complete_mask/total_num_phase1_subjs*100 + 
         total_num_phase1_partial_mask/total_num_phase1_subjs*100 + 
         total_num_phase1_queue_mask/total_num_phase1_subjs*100 + 
         phase1_num_failed_mask/total_num_phase1_subjs*100 + 
         phase1_num_na_mask/total_num_phase1_subjs*100]

phase1_num_na_mask_unet = total_num_phase1_queue_partial_unet + phase1_num_failed_partial_unet + phase1_num_na_partial_unet
phase1_num_failed_mask_unet = total_num_phase1_subjs - \
                            (total_num_phase1_complete_mask_unet + total_num_phase1_queue_mask_unet + total_num_phase1_partial_mask_unet + phase1_num_na_mask_unet)

if phase1_num_failed_mask_unet < 0:
    total_num_phase1_queue_mask_unet += phase1_num_failed_mask_unet
    phase1_num_failed_mask_unet = 0

copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['PartialLabelMap Unet --> Masking Unet',
     total_num_phase1_complete_mask_unet,
     total_num_phase1_partial_mask_unet,
     total_num_phase1_queue_mask_unet,
     phase1_num_failed_mask_unet,
     phase1_num_na_mask_unet,
     total_num_phase1_complete_mask_unet + total_num_phase1_queue_mask_unet + total_num_phase1_partial_mask_unet + phase1_num_failed_mask_unet + phase1_num_na_mask_unet,
     
     total_num_phase1_complete_mask_unet/total_num_phase1_subjs*100, 
     total_num_phase1_partial_mask_unet/total_num_phase1_subjs*100, 
     total_num_phase1_queue_mask_unet/total_num_phase1_subjs*100, 
     phase1_num_failed_mask_unet/total_num_phase1_subjs*100,
     phase1_num_na_mask_unet/total_num_phase1_subjs*100,
     
     total_num_phase1_complete_mask_unet/total_num_phase1_subjs*100 + 
         total_num_phase1_partial_mask_unet/total_num_phase1_subjs*100 + 
         total_num_phase1_queue_mask_unet/total_num_phase1_subjs*100 + 
         phase1_num_failed_mask_unet/total_num_phase1_subjs*100 + 
         phase1_num_na_mask_unet/total_num_phase1_subjs*100]

phase1_num_na_supervoxel = total_num_phase1_queue_mask + phase1_num_failed_mask + phase1_num_na_mask
phase1_num_failed_supervoxel = total_num_phase1_subjs - \
                            (total_num_phase1_complete_supervoxel + total_num_phase1_queue_supervoxel + total_num_phase1_partial_supervoxel + phase1_num_na_supervoxel)

if phase1_num_failed_supervoxel < 0:
    total_num_phase1_queue_supervoxel += phase1_num_failed_supervoxel
    phase1_num_failed_supervoxel = 0
    
copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['Isotropic --> SuperVoxels',
     total_num_phase1_complete_supervoxel,
     total_num_phase1_partial_supervoxel, 
     total_num_phase1_queue_supervoxel, 
     phase1_num_failed_supervoxel, 
     phase1_num_na_supervoxel,
     
     total_num_phase1_complete_supervoxel + 
         total_num_phase1_partial_supervoxel + 
         total_num_phase1_queue_supervoxel + 
         phase1_num_failed_supervoxel + 
         phase1_num_na_supervoxel, 
     
     total_num_phase1_complete_supervoxel/total_num_phase1_subjs*100,
     total_num_phase1_partial_supervoxel/total_num_phase1_subjs*100,
     total_num_phase1_queue_supervoxel/total_num_phase1_subjs*100,
     phase1_num_failed_supervoxel/total_num_phase1_subjs*100,
     phase1_num_na_supervoxel/total_num_phase1_subjs*100,     
     
     total_num_phase1_complete_supervoxel/total_num_phase1_subjs*100 + 
         total_num_phase1_partial_supervoxel/total_num_phase1_subjs*100 + 
         total_num_phase1_queue_supervoxel/total_num_phase1_subjs*100 + 
         phase1_num_failed_supervoxel/total_num_phase1_subjs*100 + 
         phase1_num_na_supervoxel/total_num_phase1_subjs*100]

phase1_num_na_fHOG = total_num_phase1_queue_mask + phase1_num_failed_mask + phase1_num_na_mask
phase1_num_failed_fHOG = total_num_phase1_subjs - \
                            (total_num_phase1_complete_fHOG + total_num_phase1_queue_fHOG + total_num_phase1_partial_fHOG + phase1_num_na_fHOG)

if phase1_num_failed_fHOG < 0:
    total_num_phase1_queue_fHOG += phase1_num_failed_fHOG
    phase1_num_failed_fHOG = 0
    
copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['Isotropic --> fHOG',
     total_num_phase1_complete_fHOG,
     total_num_phase1_partial_fHOG, 
     total_num_phase1_queue_fHOG, 
     phase1_num_failed_fHOG,
     phase1_num_na_fHOG,
     total_num_phase1_complete_fHOG + total_num_phase1_queue_fHOG + total_num_phase1_partial_fHOG + phase1_num_failed_fHOG + phase1_num_na_fHOG,
     
     total_num_phase1_complete_fHOG/total_num_phase1_subjs*100,
     total_num_phase1_partial_fHOG/total_num_phase1_subjs*100,
     total_num_phase1_queue_fHOG/total_num_phase1_subjs*100,
     phase1_num_failed_fHOG/total_num_phase1_subjs*100,
     phase1_num_na_fHOG/total_num_phase1_subjs*100,
     total_num_phase1_complete_fHOG/total_num_phase1_subjs*100 + 
         total_num_phase1_partial_fHOG/total_num_phase1_subjs*100 + 
         total_num_phase1_queue_fHOG/total_num_phase1_subjs*100 + 
         phase1_num_failed_fHOG/total_num_phase1_subjs*100 + 
         phase1_num_na_fHOG/total_num_phase1_subjs*100]

phase1_num_na_fHOG_hist = total_num_phase1_queue_fHOG + phase1_num_failed_fHOG + phase1_num_na_fHOG
phase1_num_failed_fHOG_hist = total_num_phase1_subjs - \
                            (total_num_phase1_complete_fHOG_hist + total_num_phase1_queue_fHOG_hist + total_num_phase1_partial_fHOG_hist + phase1_num_na_fHOG_hist)

if phase1_num_failed_fHOG_hist < 0:
    total_num_phase1_queue_fHOG_hist += phase1_num_failed_fHOG_hist
    phase1_num_failed_fHOG_hist = 0
    
copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['fHOG --> fHOG Hist',
     total_num_phase1_complete_fHOG_hist,
     total_num_phase1_partial_fHOG_hist, 
     total_num_phase1_queue_fHOG_hist, 
     phase1_num_failed_fHOG_hist,
     phase1_num_na_fHOG_hist,
     total_num_phase1_complete_fHOG_hist + total_num_phase1_queue_fHOG_hist + total_num_phase1_partial_fHOG_hist + phase1_num_failed_fHOG_hist + phase1_num_na_fHOG_hist,
     
     total_num_phase1_complete_fHOG_hist/total_num_phase1_subjs*100,
     total_num_phase1_partial_fHOG_hist/total_num_phase1_subjs*100,
     total_num_phase1_queue_fHOG_hist/total_num_phase1_subjs*100,
     phase1_num_failed_fHOG_hist/total_num_phase1_subjs*100,
     phase1_num_na_fHOG_hist/total_num_phase1_subjs*100,
     total_num_phase1_complete_fHOG_hist/total_num_phase1_subjs*100 + 
         total_num_phase1_partial_fHOG_hist/total_num_phase1_subjs*100 + 
         total_num_phase1_queue_fHOG_hist/total_num_phase1_subjs*100 + 
         phase1_num_failed_fHOG_hist/total_num_phase1_subjs*100 + 
         phase1_num_na_fHOG_hist/total_num_phase1_subjs*100]

phase1_num_na_haralick_feature = total_num_phase1_queue_supervoxel + phase1_num_failed_supervoxel + phase1_num_na_supervoxel
phase1_num_failed_haralick_feature = total_num_phase1_subjs - \
                            (total_num_phase1_complete_haralick_feature + total_num_phase1_queue_haralick_feature + total_num_phase1_partial_haralick_feature + phase1_num_na_haralick_feature)

if phase1_num_failed_haralick_feature < 0:
    total_num_phase1_queue_haralick_feature += phase1_num_failed_haralick_feature
    phase1_num_failed_haralick_feature = 0
    
copd_phase1_steps_df.loc[copd_phase1_steps_df.shape[0]] = \
    ['SuperVoxels --> Haralick Features',
     total_num_phase1_complete_haralick_feature,
     total_num_phase1_partial_haralick_feature, 
     total_num_phase1_queue_haralick_feature, 
     phase1_num_failed_haralick_feature,
     phase1_num_na_haralick_feature,
     total_num_phase1_complete_haralick_feature + total_num_phase1_queue_haralick_feature + total_num_phase1_partial_haralick_feature + phase1_num_failed_haralick_feature + phase1_num_na_haralick_feature,
     
     total_num_phase1_complete_haralick_feature/total_num_phase1_subjs*100,
     total_num_phase1_partial_haralick_feature/total_num_phase1_subjs*100,
     total_num_phase1_queue_haralick_feature/total_num_phase1_subjs*100,
     phase1_num_failed_haralick_feature/total_num_phase1_subjs*100,
     phase1_num_na_fHOG_hist/total_num_phase1_subjs*100,
     total_num_phase1_complete_haralick_feature/total_num_phase1_subjs*100 + 
         total_num_phase1_partial_haralick_feature/total_num_phase1_subjs*100 + 
         total_num_phase1_queue_haralick_feature/total_num_phase1_subjs*100 + 
         phase1_num_failed_haralick_feature/total_num_phase1_subjs*100 + 
         phase1_num_na_haralick_feature/total_num_phase1_subjs*100]

print('total_num_phase1_subjs:',total_num_phase1_subjs)        

print('###############################################')      
print('total_num_phase1_complete_niix:',total_num_phase1_complete_niix)
print('total_num_phase1_complete_iso:',total_num_phase1_complete_iso)
print('total_num_phase1_complete_partial:',total_num_phase1_complete_partial)
print('total_num_phase1_complete_partial_unet:',total_num_phase1_complete_partial_unet)
print('total_num_phase1_complete_lobe_unet:',total_num_phase1_complete_lobe_unet)
print('total_num_phase1_complete_mask:',total_num_phase1_complete_mask)
print('total_num_phase1_complete_mask_unet:',total_num_phase1_complete_mask_unet)
print('total_num_phase1_complete_supervoxel:',total_num_phase1_complete_supervoxel)
print('total_num_phase1_complete_fHOG:',total_num_phase1_complete_fHOG)
print('total_num_phase1_complete_fHOG_hist:',total_num_phase1_complete_fHOG_hist)
print('total_num_phase1_complete_haralick_feature:',total_num_phase1_complete_haralick_feature)

print('###############################################')      
print('total_num_phase1_queue_niix:',total_num_phase1_queue_niix)
print('total_num_phase1_queue_iso:',total_num_phase1_queue_iso)
print('total_num_phase1_queue_partial:',total_num_phase1_queue_partial)
print('total_num_phase1_queue_partial_unet:',total_num_phase1_queue_partial_unet)
print('total_num_phase1_queue_lobe_unet:',total_num_phase1_queue_lobe_unet)
print('total_num_phase1_queue_mask:',total_num_phase1_queue_mask)
print('total_num_phase1_queue_mask_unet:',total_num_phase1_queue_mask_unet)
print('total_num_phase1_queue_supervoxel:',total_num_phase1_queue_supervoxel)
print('total_num_phase1_queue_fHOG:',total_num_phase1_queue_fHOG)     
print('total_num_phase1_queue_fHOG_hist:',total_num_phase1_queue_fHOG_hist)     
print('total_num_phase1_queue_haralick_feature:',total_num_phase1_queue_haralick_feature)

print('###############################################')      
print('total_num_phase1_partial_niix:',total_num_phase1_partial_niix)
print('total_num_phase1_partial_iso:',total_num_phase1_partial_iso)
print('total_num_phase1_partial_partial:',total_num_phase1_partial_partial)
print('total_num_phase1_partial_partial_unet:',total_num_phase1_partial_partial_unet)
print('total_num_phase1_partial_lobe_unet:',total_num_phase1_partial_lobe_unet)
print('total_num_phase1_partial_mask:',total_num_phase1_partial_mask)
print('total_num_phase1_partial_mask_unet:',total_num_phase1_partial_mask_unet)
print('total_num_phase1_partial_supervoxel:',total_num_phase1_partial_supervoxel)
print('total_num_phase1_partial_fHOG:',total_num_phase1_partial_fHOG)     
print('total_num_phase1_partial_fHOG_hist:',total_num_phase1_partial_fHOG_hist)
print('total_num_phase1_partial_haralick_feature:',total_num_phase1_partial_haralick_feature)

print('')
print('Dicom --> Niix: Dicom to Niix Conversion Step')
print('Niix --> Isotropic: Niix to Isotropic Conversion Step')
print('Isotropic --> PartialLabelMap: Lung Segmentation on Isotropic Image Step')
print('Isotropic --> PartialLabelMap Unet: Lung Segmentation on Isotropic Image by Unet Model Step')
print('Isotropic --> LobeSegmentation Unet: Lobe Segmentation on Isotropic Image by Unet Model Step')
print('PartialLabelMap --> Masking: Create a Masking Image from Lung Segmentation Step')
print('PartialLabelMap Unet --> Masking Unet: Create a Masking Image from Unet-Model Lung Segmentation Step')
print('Isotropic --> fHOGs: Generate a fHOG Image from Isotropic One Step')
print('fHOGs --> fHOG Hist: Extract a fHOG Histogram Features from fHOG Image Step')
print('SuperVoxels --> Haralick Features: Extract a Haralick Features from SuperVoxels Image Step')

copd_phase1_steps_df

total_num_phase1_subjs: 5882
###############################################
total_num_phase1_complete_niix: 5690
total_num_phase1_complete_iso: 4299
total_num_phase1_complete_partial: 0
total_num_phase1_complete_partial_unet: 4299
total_num_phase1_complete_lobe_unet: 4299
total_num_phase1_complete_mask: 0
total_num_phase1_complete_mask_unet: 0
total_num_phase1_complete_supervoxel: 0
total_num_phase1_complete_fHOG: 0
total_num_phase1_complete_fHOG_hist: 0
total_num_phase1_complete_haralick_feature: 0
###############################################
total_num_phase1_queue_niix: 0
total_num_phase1_queue_iso: 0
total_num_phase1_queue_partial: 0
total_num_phase1_queue_partial_unet: 0
total_num_phase1_queue_lobe_unet: 0
total_num_phase1_queue_mask: 0
total_num_phase1_queue_mask_unet: 0
total_num_phase1_queue_supervoxel: 0
total_num_phase1_queue_fHOG: 0
total_num_phase1_queue_fHOG_hist: 0
total_num_phase1_queue_haralick_feature: 0
###############################################
total_num_phas

,step,#img complete,#img partial,#img queue,#img failed,#img NA,#total,%complete,%partial,%queue,%failed,%NA,%total
0,Dicom --> Niix,5690,0,0,192,0,5882,96.735804,0.0,0.0,3.264196,0.000000,100.0
1,Niix --> Isotropic,4299,0,0,1391,192,5882,73.087385,0.0,0.0,23.648419,3.264196,100.0
2,Isotropic --> PartialLabelMap,0,0,0,4299,1583,5882,0.000000,0.0,0.0,73.087385,26.912615,100.0
3,Isotropic --> PartialLabelMap Unet,4299,0,0,0,1583,5882,73.087385,0.0,0.0,0.000000,26.912615,100.0
4,Isotropic --> LobeSegmentation Unet,4299,0,0,0,1583,5882,73.087385,0.0,0.0,0.000000,26.912615,100.0
5,PartialLabelMap --> Masking,0,0,0,0,5882,5882,0.000000,0.0,0.0,0.000000,100.000000,100.0
6,PartialLabelMap Unet --> Masking Unet,0,0,0,4299,1583,5882,0.000000,0.0,0.0,73.087385,26.912615,100.0
7,Isotropic --> SuperVoxels,0,0,0,0,5882,5882,0.000000,0.0,0.0,0.000000,100.000000,100.0
8,Isotropic --> fHOG,0,0,0,0,5882,5882,0.000000,0.0,0.0,0.000000,100.000000,100.0
9,fHOG --> fHOG Hist,0,0,0,0,5882,5882,0.000000,0.0,0.0,0.000000,100.000000,100.0
